###  Training


In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 

env = env.unwrapped



import tensorflow as tf
import policy_gradient

In [ ]:
# create the tf session
sess = tf.InteractiveSession()

# Input
num_actions=3  # same as # of stocks
Variables=3
obs_dim=num_actions*Variables
NumOfLayers=3
LR="Adam"
actFunc="relu"
regulizer="l2"
regulizerScale=0.01


pg = policy_gradient.PolicyGradient(sess, obs_dim=obs_dim, 
                                    num_actions=num_actions,
                                    NumOfLayers=NumOfLayers, 
                                    LR=LR,
                                    actFunc=actFunc,
                                    learning_rate=1e-2,
                                    regulizer =regulizer,
                                    regulizerScale=regulizerScale
                                   )

# and now let's train it and evaluate its progress.  NB: this could take some time...
direc="/Users/andrewplaate/mlp3/SavedModels/"
load_model=True
df,sf = pg.train_model(env, episodes=25001, log_freq=100, load_model=load_model,model_dir = direc)#, load_model=True)


# Testing

In [1]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env1 = gym.make('testing-v0')
#env.time_cost_bps = 0 # 

env1 = env1.unwrapped



import tensorflow as tf
import policy_gradient

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_trading.envs.Testing_Env.TestingEnv'>' has deprecated methods. Compatibility code invoked.


/Users/andrewplaate/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:policy_gradient:policy_gradient logger started.


In [2]:
# create the tf session
sess = tf.InteractiveSession()

# Input
num_actions=3  # same as # of stocks
Variables=3
obs_dim=num_actions*Variables
NumOfLayers=3
LR="Adam"
actFunc="relu"
regulizer="l2"
regulizerScale=0.01


pg = policy_gradient.PolicyGradient(sess, obs_dim=obs_dim, 
                                    num_actions=num_actions,
                                    NumOfLayers=NumOfLayers, 
                                    LR=LR,
                                    actFunc=actFunc,
                                    learning_rate=1e-2,
                                    regulizer =regulizer,
                                    regulizerScale=regulizerScale
                                   )

# and now let's train it and evaluate its progress.  NB: this could take some time...
direc="/Users/andrewplaate/mlp3/SavedModels/"
load_model=True
df,sf = pg.test_model(env1, episodes=1, log_freq=100, load_model=load_model,model_dir = direc)#, load_model=True)


/Users/andrewplaate/mlp3/SavedModels/model.ckpt-1
INFO:tensorflow:Restoring parameters from /Users/andrewplaate/mlp3/SavedModels/model.ckpt-1


INFO:tensorflow:Restoring parameters from /Users/andrewplaate/mlp3/SavedModels/model.ckpt-1


Model restored from  /Users/andrewplaate/mlp3/SavedModels/model.ckpt-1
0 [-2.]
0 [ 0.12559406]
0 [ 28.80303887]
0 [-0.92639857]
0 [ 42.68522082]
0 [-0.86840676]
0 [-0.86866808]
0 [ 7.36528967]
0 [-1.78631567]
0 [-1.05999519]
0 [ 36.28908788]
0 [-0.89244757]
0 [ 4.19291221]
0 [-0.75900497]
0 [ 6.0692826]
0 [-0.65211808]
0 [-4.8398655]
0 [ 0.34545623]
0 [ 0.97649722]
0 [ 1.26118545]
0 [-0.98472863]
0 [ 0.02469532]
0 [-0.85683007]
0 [-0.58366264]
0 [ 235.51259891]
0 [-0.74742108]
0 [-0.87112721]
0 [ 0.57433141]
0 [ 30.35992051]
0 [-0.82048061]
0 [ 8.35946446]
0 [ 0.14442003]
0 [-0.72909303]
0 [ 1.79728699]
0 [ 0.34153304]
0 [-0.91218252]
0 [-0.08022292]
0 [ 11.25774985]
0 [ 0.99242715]
0 [-0.98718422]
0 [ 0.26458538]
0 [ 35.00149627]
0 [-0.99211032]
0 [ 84.9298569]
0 [ 0.00932049]
0 [ 0.08400085]
0 [ 0.3652234]
0 [-0.95712926]
0 [ 21.56089121]
0 [-0.9487687]
0 [-0.3826257]
0 [ 27.67716472]
0 [-0.89554801]
0 [ 9.10840145]
0 [ 0.0262893]
0 [-0.98833603]
0 [ 88.67877633]
0 [-0.92782358]
0 [ 

INFO:policy_gradient:year #     0, mean reward:      nan, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.00331441]
0 [ 0.01300582]
0 [-0.91962485]
0 [ 0.09527007]
0 [-0.18443938]
0 [ 12.61001532]
0 [ 0.00457138]
0 [-0.97515302]
0 [ 38.51883444]
0 [-0.98011111]
0 [ 47.83585263]
0 [ 0.0513781]
0 [-0.973605]
0 [ 36.15138882]
0 [ 0.02514981]
0 [ 0.04054299]
0 [-0.98968957]
0 [ 93.33610919]
0 [ 0.00325125]
0 [-0.99538333]
0 [ 192.68460243]
0 [ 0.05416214]
0 [-0.94520414]
0 [ 18.32032506]
0 [ 0.04226226]
0 [ 0.01287941]
0 [-0.92888699]
0 [-0.65304792]
0 [ 41.43220478]
0 [-0.96431939]
0 [-0.16475074]
0 [ 0.72199952]
0 [ 10.10587792]
0 [ 0.66940732]
0 [-0.85637863]
0 [ 6.01030519]
0 [  5.27598266e-05]
0 [ nan]


### Results

Policy gradients beat the trading game!  That said, it doesn't work every time and it seems, looking at the charts below, as though it's a bit of a lucky thing.  But luck counts in the trading game as in life!


In [ ]:
sf['net'] = sf.simror - sf.mktror
#sf.net.plot()
sf.net.expanding().mean().plot()
sf.net.rolling(100).mean().plot()

In [ ]:
sf.net.rolling(100).mean().tail()